In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import os

import pandas as pd
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v7 import CNN_v7

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn = CNN_v7((480, 640, 4), split_into_parts=20)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v7/weights/"

def get_weights_paths(prefix):
    for directory, subdirectories, files in os.walk(prefix):
        yield from (os.path.join(directory, file) for file in files)

model_names = get_weights_paths(prefix)
model_names = list(model_names)

In [7]:
models = [CNN_v7((480, 640, 4), split_into_parts=20) for i in range(len(model_names))]

In [8]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=2, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.7203 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 905ms/step - loss: 0.7245 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.7265 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 875ms/step - loss: 0.7249 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 882ms/step - loss: 0.7208 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.7212 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 863ms/step - loss: 0.7245 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 863ms/step - loss: 0.7207 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 865ms/step - loss: 0.7258 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 919ms/step - loss: 0.7225 - accuracy: 1.0000
Epoch 1/1
1/1 [========================

In [9]:
def as_rows(results):
    for name, cut_off_point_to_accuracy_with_difference in results.items():
        yield from ((f"{name}_{cut_off_point}", accuracy, counts - count_test) for cut_off_point, (accuracy, counts) in cut_off_point_to_accuracy_with_difference.items())

rows = as_rows(results)
rows = list(rows)

In [10]:
def calculate_mae_and_mse(rows):
    for *attributes, difference in rows:
        difference_mae = mae(difference)
        difference_mse = mse(difference)
        
        yield (*attributes, difference_mae, difference_mse)

mae_and_mse = calculate_mae_and_mse(rows)
mae_and_mse = list(mae_and_mse)

In [11]:
frame = pd.DataFrame(mae_and_mse, columns=["model", "accuracy", "mae", "mse"])
frame

,model,accuracy,mae,mse
0,CNN/CNN_v7/weights/cnn_v7_1e_6_5_0.1,0.000000,98.866667,9985.153333
1,CNN/CNN_v7/weights/cnn_v7_1e_6_5_0.2,0.708894,46.946667,2324.166667
2,CNN/CNN_v7/weights/cnn_v7_1e_6_5_0.30000000000...,22.314374,24.850000,680.183333
3,CNN/CNN_v7/weights/cnn_v7_1e_6_5_0.4,61.513665,11.573333,166.240000
4,CNN/CNN_v7/weights/cnn_v7_1e_6_5_0.5,87.298994,3.800000,22.786667
...,...,...,...,...
355,CNN/CNN_v7/weights/cnn_v7_1e_6_44_0.5,86.449123,4.166667,25.533333
356,CNN/CNN_v7/weights/cnn_v7_1e_6_44_0.6000000000...,76.150920,7.460000,69.146667
357,CNN/CNN_v7/weights/cnn_v7_1e_6_44_0.7000000000...,66.194377,10.580000,128.946667
358,CNN/CNN_v7/weights/cnn_v7_1e_6_44_0.8,54.677781,14.113333,219.373333


In [12]:
frame.loc[frame.mae == frame.mae.min()]

,model,accuracy,mae,mse
165,CNN/CNN_v7/weights/cnn_v7_1e_6_23_0.4,90.136901,2.996667,14.096667


In [13]:
frame.loc[frame.mse == frame.mse.min()]

,model,accuracy,mae,mse
165,CNN/CNN_v7/weights/cnn_v7_1e_6_23_0.4,90.136901,2.996667,14.096667


In [14]:
frame.loc[frame.accuracy == frame.accuracy.max()]

,model,accuracy,mae,mse
165,CNN/CNN_v7/weights/cnn_v7_1e_6_23_0.4,90.136901,2.996667,14.096667
